In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from utils.import_utils import *
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from src.feature_processing import *
from src.item_processing import  *
from src.utils.stats_utils import *
import plotly.express as px
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
with initialize(config_path='../configuration', version_base='1.1'):
    config = compose(config_name='main.yaml')

In [ ]:
features_class = ItemFeatureProcessing(config)

In [ ]:
df_item = features_class.df_item
self = features_class

# Answer Position ITEM Level Processing


In [ ]:
feature_name = 'f__answer_position'
score_name = 's__answer_position'
df = df_item[~pd.isnull(df_item[feature_name])].copy()
# Select only those variables that have at least three distinct values and more than one hundred records

variables =  self.filter_variable_name_by_frequency(df, feature_name, frequency=100, min_unique_values=3)




##### Plot the Relative Position of each variable

In [ ]:
for index_range in range(0, len(variables), 50):
    plt.figure(figsize=(15, 6))
    sns.boxplot(df[df['variable_name'].isin(variables[index_range:index_range+50])],x='variable_name', y=feature_name)
    plt.xticks(rotation=90)
    plt.show()

1. Calculate the entropy for each variable_name per responsible, i.e., one entropy value for each responsible and variable_name.
2. Entropy is calculated only for those varible_name of each responsible that have at least 10 times the number of possible values of the variable_name
3. Mark as "anomalous" any entropy value that is lower than the 50% of the median entropy value
4. Plot the entropy value for each variable name by responsible and colour in red those values that have very low entropy

In [ ]:
for var in variables:

    mask = (df['variable_name']==var) 
    unique_values = df[mask][feature_name].nunique()
    entropy_df = df[mask].groupby('responsible')[feature_name].apply(calculate_entropy,unique_values=unique_values, min_record_sample=5).copy()
    entropy_df = entropy_df.reset_index()
    entropy_df = entropy_df[~pd.isnull(entropy_df[feature_name])]
    
    if entropy_df.shape[0] > 0:
        entropy_df.sort_values(feature_name, inplace=True, ascending=False)

        median_value = entropy_df[feature_name].median()
        entropy_df[score_name] = entropy_df[feature_name].apply(lambda x: 1 if x< median_value-50/100*median_value else 0)

        anomaly_df = entropy_df[entropy_df[score_name] == 1]
        no_anomaly_df = entropy_df[entropy_df[score_name] == 0]
        plt.bar(no_anomaly_df['responsible'], no_anomaly_df[feature_name], color='blue', label='No Anomaly')
        plt.bar(anomaly_df['responsible'], anomaly_df[feature_name], color='red', label='Anomaly')
        plt.xticks(rotation=90)
        plt.title(var)
        plt.show()


In [ ]:
for var in variables:
    mask = (df['variable_name']==var)
    unique_values = df[mask][feature_name].nunique()
    entropy_df = df[mask].groupby('responsible')[feature_name].apply(calculate_entropy,unique_values=unique_values)
    entropy_df = entropy_df.reset_index()
    entropy_df = entropy_df[~pd.isnull(entropy_df[feature_name])]
    
    if entropy_df.shape[0] > 0:
        entropy_df.sort_values(feature_name, inplace=True, ascending=False)

        median_value = entropy_df[feature_name].median()

        median_value = entropy_df[feature_name].median()
        entropy_df[score_name] = entropy_df[feature_name].apply(lambda x: 1 if x < median_value - 50/100 * median_value else 0)
        df.loc[mask,score_name] = df[mask]['responsible'].map(entropy_df.set_index('responsible')[score_name])
    # 

# Answer Position Responsible Level Processing

In [ ]:
data = df.groupby(['responsible','variable_name']).agg({score_name: 'mean', feature_name:'count'})
data = data.reset_index()
entropy_ = data.groupby('responsible')[score_name].mean()
entropy_.plot(kind='bar')
plt.title('entropy__answer_position')
plt.show()